In [1]:
import numpy as np

# Install Tensorflow Hub and Database
!pip install tensorflow_hub
!pip install tensorflow_database

#TensorFlow Packages
import tensorflow as tf
import tensorflow_hub as tfhub
import tensorflow_datasets as tfds

ERROR: Could not find a version that satisfies the requirement tensorflow_database (from versions: none)
ERROR: No matching distribution found for tensorflow_database


In [2]:
# To load movie reviews from tensorflow and loading first 60 % data (0% to 60%)
# and then taking rest 40 % from 60 % to 100 %
train_data, validation_data, test_data = tfds.load(name="imdb_reviews", split=('train[:60%]','train[60%:]' ,'test'),as_supervised=True)


# train_example_batch has list of all Movie Reviews
# train_label_batch has output 1-> Positive Review, 0 -> Negative Review
train_example_batch, train_label_batch = next (iter(train_data.batch(10)))


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.Y6XW9O_1.0.0/imdb_reviews-train.tfrecor…

Generating test examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.Y6XW9O_1.0.0/imdb_reviews-test.tfrecord…

Generating unsupervised examples...: 0 examples [00:00, ? examples/s]

Shuffling /root/tensorflow_datasets/imdb_reviews/plain_text/incomplete.Y6XW9O_1.0.0/imdb_reviews-unsupervised.…

Dataset imdb_reviews downloaded and prepared to /root/tensorflow_datasets/imdb_reviews/plain_text/1.0.0. Subsequent calls will reuse this data.


In [3]:
train_data

<_PrefetchDataset element_spec=(TensorSpec(shape=(), dtype=tf.string, name=None), TensorSpec(shape=(), dtype=tf.int64, name=None))>

In [4]:
train_example_batch

<tf.Tensor: shape=(10,), dtype=string, numpy=
array([b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it.",
       b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tired, being warm and comfortable on the sette and having just eaten a lot. However on this occasion I fell 

In [5]:
train_label_batch

<tf.Tensor: shape=(10,), dtype=int64, numpy=array([0, 0, 0, 1, 1, 1, 0, 0, 0, 0])>

In [6]:
print("GPU is ", "available" if tf.config.experimental.list_physical_devices("GPU") else "NOT AVAILABLE")

GPU is  NOT AVAILABLE


In [7]:
embedding = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

#Taking one of those ready-made models and using it as a layer inside my own Keras/TensorFlow model
hub_layer = tfhub.KerasLayer(embedding, input_shape=[] , dtype = tf.string, trainable=True)

In [8]:
hub_layer(train_example_batch[:3])

# Creating sequential model and adding 'hub_layer' to our model

# A Dense layer in Keras is just a fully connected layer in a neural network to exchange data (information)

<tf.Tensor: shape=(3, 20), dtype=float32, numpy=
array([[ 1.765786  , -3.882232  ,  3.9134233 , -1.5557289 , -3.3362343 ,
        -1.7357955 , -1.9954445 ,  1.2989551 ,  5.081598  , -1.1041286 ,
        -2.0503852 , -0.72675157, -0.65675956,  0.24436149, -3.7208383 ,
         2.0954835 ,  2.2969332 , -2.0689783 , -2.9489717 , -1.1315987 ],
       [ 1.8804485 , -2.5852382 ,  3.4066997 ,  1.0982676 , -4.056685  ,
        -4.891284  , -2.785554  ,  1.3874227 ,  3.8476458 , -0.9256538 ,
        -1.896706  ,  1.2113281 ,  0.11474707,  0.76209456, -4.8791065 ,
         2.906149  ,  4.7087674 , -2.3652055 , -3.5015898 , -1.6390051 ],
       [ 0.71152234, -0.6353217 ,  1.7385626 , -1.1168286 , -0.5451594 ,
        -1.1808156 ,  0.09504455,  1.4653089 ,  0.66059524,  0.79308075,
        -2.2268345 ,  0.07446612, -1.4075904 , -0.70645386, -1.907037  ,
         1.4419787 ,  1.9551861 , -0.42660055, -2.8022065 ,  0.43727064]],
      dtype=float32)>

In [9]:
from tensorflow import keras

print("Sequential from:", tf.keras.Sequential)
print("Hub Layer from:", tfhub.KerasLayer)
print("Standalone keras Sequential:", keras.Sequential)


# Input: a sentence (string)
# Hub embedding layer → turns it into a dense vector (numbers) which neural network can understand
# Dense layers → learn patterns from those vectors
# Without Embedding Layer, the model would not know how to handle text like "good movie" vs "bad movie"
# Output: 0(bad movie) or 1(good movie)

# Custom Layer wrapping TF Hub embedding for the URL
class HubEmbeddingLayer(keras.layers.Layer):
    def __init__(self, hub_url, **kwargs):
        super(HubEmbeddingLayer, self).__init__(**kwargs)
        # Create hub embedding layer
        self._hub_layer = tfhub.KerasLayer(
            hub_url,
            input_shape=[],
            dtype=tf.string,
            trainable=True
        )

    def call(self, inputs):
        return self._hub_layer(inputs)

# Our embedding URL
embedding_url = "https://tfhub.dev/google/tf2-preview/gnews-swivel-20dim/1"

# Build Sequential model
model = keras.Sequential([
    keras.layers.Input(shape=[], dtype=tf.string),
    HubEmbeddingLayer(embedding_url),
    keras.layers.Dense(64, activation='relu'),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(32, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

Sequential from: <class 'keras.src.models.sequential.Sequential'>
Hub Layer from: <class 'tensorflow_hub.keras_layer.KerasLayer'>
Standalone keras Sequential: <class 'keras.src.models.sequential.Sequential'>


In [10]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ hub_embedding_layer             │ (None, 20)             │             0 │
│ (HubEmbeddingLayer)             │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │         1,344 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 64)             │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 3,457 (13.50 KB)

 Trainable params: 3,457 (13.50 KB)

 Non-trainable params: 0 (0.00 B)

In [11]:
# compile() cell -> Call as per requirement
# Compile the model

# Adam ->  a popular optimizer that adjusts learning rate automatically (fast + reliable)

# Since we are predicting 0 or 1 [Positive review (1) vs Negative review (0)],
#(binary classification), binary crossentropy is the best choice

# Metrics -> Our goal : Accuracy
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [12]:

# Training the model

# verbose=1 → shows a progress bar per epoch with loss and accuracy values
# shuffle -> randomizes order of data

history = model.fit(train_data.shuffle(10000).batch(512),
                    epochs=20,
                    validation_data=validation_data.batch(512),
                    verbose=1)

# Epoch -> An epoch means one full pass of the training data through the model.

# If we have 1000 training samples:
# In 1 epoch, the model will see all 1000 samples once
# In 10 epochs, it will see the entire dataset 10 times

Epoch 1/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 8s 164ms/step - accuracy: 0.5243 - loss: 0.7724 - val_accuracy: 0.6447 - val_loss: 0.6335
Epoch 2/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 6s 55ms/step - accuracy: 0.6190 - loss: 0.6546 - val_accuracy: 0.6758 - val_loss: 0.6078
Epoch 3/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.6562 - loss: 0.6269 - val_accuracy: 0.6911 - val_loss: 0.5929
Epoch 4/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.6773 - loss: 0.6074 - val_accuracy: 0.6967 - val_loss: 0.5823
Epoch 5/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - accuracy: 0.6748 - loss: 0.6038 - val_accuracy: 0.7050 - val_loss: 0.5746
Epoch 6/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 3s 81ms/step - accuracy: 0.6908 - loss: 0.5886 - val_accuracy: 0.7108 - val_loss: 0.5696
Epoch 7/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 62ms/step - accuracy: 0.6929 - loss: 0.5850 - val_accuracy: 0.7106 - val_loss: 0.5676
Epoch 8/20
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 56ms/step - accuracy: 0.6912 - loss: 0.5847 - val_accuracy: 0.7140 - 

In [13]:
# NOTE -> Call compile() first  before running this cell whenever we want to see accuracy

# verbose=2 → shows only one line per epoch (no progress bar, only information)
results = model.evaluate(test_data.batch(100), verbose=2)

# metrics_names -> list of metrics which model is tracking (loss,accuracy)
for name, value in zip(model.metrics_names, results):
  print("%s: %.3f" % (name, value))

# Accuracy obtained is around 72 %

250/250 - 4s - 14ms/step - accuracy: 0.7210 - loss: 0.5524
loss: 0.552
compile_metrics: 0.721
